In [1]:
!pip install wrds 
import wrds 
import pandas as pd

In [2]:
db = wrds.Connection()

Loading library list...
Done


In [4]:
query = """
SELECT 
    acusip,
    dateann,
    master_deal_no,
    tmanames,
    amanames,
    attitude,
    form,
    tsicp
FROM tr_sdc_ma.wrds_ma_details
WHERE acusip IN ('532457', '67066G')
  AND dateann >= '2020-01-01';
"""

df = db.raw_sql(query)


In [5]:
df.head()


,acusip,dateann,master_deal_no,tmanames,amanames,attitude,form,tsicp
0,532457,2020-01-10,3500732020,Dermira Inc,Eli Lilly & Co,Friendly,Merger,2834
1,532457,2020-03-31,3549596040,Sitryx Therapeutics Ltd,Eli Lilly & Co,Friendly,Acq. Part. Int.,8731
2,532457,2020-10-15,3648372020,Disarm Therapeutics Inc,Eli Lilly & Co,Friendly,Acq. of Assets,2836
3,532457,2021-07-14,3771469020,Protomer Technologies Inc,Eli Lilly & Co,Friendly,Merger,2836
4,532457,2021-07-29,3796084020,Kumquat Biosciences,Eli Lilly & Co,Friendly,Acq. Part. Int.,8731


In [6]:
len(df)
# or equivalently:
df.shape[0]


50

In [7]:
df_div = (
    df.groupby(['acusip', 'amanames'])
      .agg(unique_target_sics=('tsicp', lambda x: sorted(x.unique().tolist())),
           target_diversification=('tsicp', 'nunique'),
           deal_count=('master_deal_no', 'nunique'))
      .reset_index()
)
df_div


,acusip,amanames,unique_target_sics,target_diversification,deal_count
0,532457,Eli Lilly & Co,"[2834, 2836, 8099, 8731]",4,26
1,67066G,NVIDIA Corp,"[3674, 3721, 7372, 7373, 7376]",5,24


In [8]:
query = """
SELECT DISTINCT tsicp, ttf_macro_desc
FROM tr_sdc_ma.wrds_ma_details
WHERE acusip IN ('532457', '67066G')
  AND dateann >= '2020-01-01';
"""
sic_desc = db.raw_sql(query)
sic_desc.head()


,tsicp,ttf_macro_desc
0,2834,Healthcare
1,2836,Healthcare
2,3674,High Technology
3,3721,Consumer Products and Services
4,7372,Consumer Products and Services


In [9]:
df_div_expanded = df_div.explode('unique_target_sics')
df_div_expanded = df_div_expanded.merge(
    sic_desc, left_on='unique_target_sics', right_on='tsicp', how='left'
)
df_div_expanded


,acusip,amanames,unique_target_sics,target_diversification,deal_count,tsicp,ttf_macro_desc
0,532457,Eli Lilly & Co,2834,4,26,2834,Healthcare
1,532457,Eli Lilly & Co,2836,4,26,2836,Healthcare
2,532457,Eli Lilly & Co,8099,4,26,8099,Healthcare
3,532457,Eli Lilly & Co,8731,4,26,8731,Healthcare
4,67066G,NVIDIA Corp,3674,5,24,3674,High Technology
5,67066G,NVIDIA Corp,3721,5,24,3721,Consumer Products and Services
6,67066G,NVIDIA Corp,7372,5,24,7372,Consumer Products and Services
7,67066G,NVIDIA Corp,7372,5,24,7372,High Technology
8,67066G,NVIDIA Corp,7373,5,24,7373,High Technology
9,67066G,NVIDIA Corp,7376,5,24,7376,High Technology
